# Initialization

In [2]:
import pymongo
import functools
import itertools
import numpy as np
import pandas as pd
from IPython.display import display, HTML

In [3]:
conn = pymongo.MongoClient('mongodb://lily.local:27017')

# Analysis

## `l1ic_size.exec`

Let's build a Pandas `DataFrame` of the execution cycle counts of all the L1I$ configuration runs that completed successfully.

In [12]:
_correct_answer = '7' # command: ../../examples/bin/sum 2 3 -5 7
_tmp_df = pd.DataFrame({
    (w.get('pipeline'),) + tuple(map(lambda y: y[1], filter(lambda x: 'fetch' in x[0], w.get('config').items()))): [w.get('stats').get('cycle'), w.get('stdout').strip() == _correct_answer]
    for w in conn.nebula.l1ic_size.find({'cmdline': {'$regex': 'sum'}})
}).T.reset_index().rename(columns={
    'level_0': 'pipeline',
    'level_1': 'nsets',
    'level_2': 'nways',
    'level_3': 'nbytesperblock',
    'level_4': 'replacement',
    0: 'cycle',
    1: 'correct_answer'
}).set_index(['pipeline', 'nsets', 'nways', 'nbytesperblock', 'replacement'])
_correct_df = _tmp_df.loc[_tmp_df.correct_answer == True].sort_index()
display(HTML(_correct_df.to_html()))
print(_correct_df.describe())

        cycle correct_answer
count      48             48
unique     47              1
top     16613           True
freq        2             48


## `watchdog.exec`

Did the `watchdog.py` component successfully detect violation of the watch condition? For the Bergamot simulations, the watch event was `commit`, and the watchdog is supposed to signal a violation if 10 or more cycles elapse between consecutive `commit` events.

In [5]:
conn.nebula.watchdog.find_one().keys()

dict_keys(['_id', 'sha', 'branch', 'exec_script', 'exitcode', 'stats', 'log', 'config', 'runpath', 'pipeline', 'script', 'cmdline', 'stdout', 'stderr', 'date', 'time'])

In [6]:
print(conn.nebula.watchdog.find_one().get('log').get('0000_watchdog.py.log'))

config : {'service': 'mainmem', 'field': 'peek_latency_in_cycles', 'val': 25}
config : {'service': 'mainmem', 'field': 'filename', 'val': '/tmp/mainmem.raw'}
config : {'service': 'mainmem', 'field': 'capacity', 'val': 4294967296}
config : {'service': 'stats', 'field': 'output_filename', 'val': '/tmp/stats.json'}
config : {'service': 'l2', 'field': 'l2_hitlatency', 'val': 1}
config : {'service': 'mainmem', 'field': 'peek_latency_in_cycles', 'val': 10}
config : {'service': 'mainmem', 'field': 'capacity', 'val': 4297967296}
config : {'service': 'watchdog', 'field': 'event_name', 'val': 'commit'}
config : {'service': 'watchdog', 'field': 'event_cycles', 'val': 10}
config : {'service': 'stats', 'field': 'output_filename', 'val': '/home/john/src/nebula/tmp/runs/994a8928-cba2-454c-a70e-065a45feccf0/stats.json'}
config : {'service': 'mainmem', 'field': 'filename', 'val': '/home/john/src/nebula/tmp/runs/994a8928-cba2-454c-a70e-065a45feccf0/mainmem.raw'}
Watchdog violation!
	state.cycle         

## `quick_*.exec`

Which of these tests failed to generate a correct result for the `negate`, `sort`, and `sum` example binaries?

In [7]:
_correct_answer = '-2' # command: ../../examples/bin/negate 2
_quick = conn.nebula.quick.find({'cmdline': {'$regex': 'negate'}})
for _q in _quick:
    _answer = _q.get('stdout').strip()
    if _correct_answer != _answer:
        print('{} ({}) : {} {}'.format(_answer, len(_answer), _q.get('pipeline'), _q.get('cmdline')))

-1328184490 (11) : pipelines/clementine python3 /home/john/src/nebula/launcher.py  --log /home/john/src/nebula/tmp/runs/51c36f39-32ed-4e36-9270-9f3fde9bffa9/log --service /home/john/src/nebula/toolbox/stats.py:localhost:-1 implementation/mainmem.py:localhost:-1 --max_cycles 100000   --config mainmem:peek_latency_in_cycles:1 mainmem:capacity:4297967296 stats:output_filename:/home/john/src/nebula/tmp/runs/51c36f39-32ed-4e36-9270-9f3fde9bffa9/stats.json mainmem:filename:/home/john/src/nebula/tmp/runs/51c36f39-32ed-4e36-9270-9f3fde9bffa9/mainmem.raw -- 10004 /home/john/src/nebula/pipelines/clementine/init.nebula ../../examples/bin/negate 2
-1328184490 (11) : pipelines/clementine python3 /home/john/src/nebula/launcher.py  --log /home/john/src/nebula/tmp/runs/506b0ef2-c392-4b90-8057-adfd857a1c8c/log --service /home/john/src/nebula/toolbox/stats.py:localhost:-1 implementation/mainmem.py:localhost:-1 --max_cycles 100000   --config mainmem:peek_latency_in_cycles:5 mainmem:capacity:4297967296 st

In [8]:
_correct_answer = '-3 2 5' # command: ../../examples/bin/sort 2 -3 5
_quick = conn.nebula.quick.find({'cmdline': {'$regex': 'sort'}})
for _q in _quick:
    _answer = _q.get('stdout').strip()
    if _correct_answer != _answer:
        print('{} ({}) : {} {}'.format(_answer, len(_answer), _q.get('pipeline'), _q.get('cmdline')))

 (0) : pipelines/bergamot python3 /home/john/src/nebula/launcher.py  --log /home/john/src/nebula/tmp/runs/e5d59334-0962-4c14-80df-cb37e3e60086/log --service /home/john/src/nebula/toolbox/stats.py:localhost:-1 implementation/mainmem.py:localhost:-1 --max_cycles 500000   --config mainmem:peek_latency_in_cycles:25 mainmem:capacity:4297967296 stats:output_filename:/home/john/src/nebula/tmp/runs/e5d59334-0962-4c14-80df-cb37e3e60086/stats.json mainmem:filename:/home/john/src/nebula/tmp/runs/e5d59334-0962-4c14-80df-cb37e3e60086/mainmem.raw -- 10003 /home/john/src/nebula/pipelines/bergamot/init.nebula ../../examples/bin/sort 2 -3 5
-1328184490 -1328184490 -1328184490 (35) : pipelines/clementine python3 /home/john/src/nebula/launcher.py  --log /home/john/src/nebula/tmp/runs/1b55622f-5e21-418b-a1d1-5054d01e7c2d/log --service /home/john/src/nebula/toolbox/stats.py:localhost:-1 implementation/mainmem.py:localhost:-1 --max_cycles 500000   --config mainmem:peek_latency_in_cycles:1 mainmem:capacity:4

In [9]:
_correct_answer = '5' # command: ../../examples/bin/sum 2 3
_quick = conn.nebula.quick.find({'cmdline': {'$regex': 'sum'}})
for _q in _quick:
    _answer = _q.get('stdout').strip()
    if _correct_answer != _answer:
        print('{} ({}) : {} {}'.format(_answer, len(_answer), _q.get('pipeline'), _q.get('cmdline')))

-1328184490 (11) : pipelines/clementine python3 /home/john/src/nebula/launcher.py  --log /home/john/src/nebula/tmp/runs/44cbe603-81e0-4f21-bc3e-dfa4a5d9ddaf/log --service /home/john/src/nebula/toolbox/stats.py:localhost:-1 implementation/mainmem.py:localhost:-1 --max_cycles 100000   --config mainmem:peek_latency_in_cycles:1 mainmem:capacity:4297967296 stats:output_filename:/home/john/src/nebula/tmp/runs/44cbe603-81e0-4f21-bc3e-dfa4a5d9ddaf/stats.json mainmem:filename:/home/john/src/nebula/tmp/runs/44cbe603-81e0-4f21-bc3e-dfa4a5d9ddaf/mainmem.raw -- 10004 /home/john/src/nebula/pipelines/clementine/init.nebula ../../examples/bin/sum 2 3
-1328184490 (11) : pipelines/clementine python3 /home/john/src/nebula/launcher.py  --log /home/john/src/nebula/tmp/runs/5b5c03ca-d4a8-47ce-8aa2-6e68e410a140/log --service /home/john/src/nebula/toolbox/stats.py:localhost:-1 implementation/mainmem.py:localhost:-1 --max_cycles 100000   --config mainmem:peek_latency_in_cycles:5 mainmem:capacity:4297967296 sta

# Scratchpad

In [10]:
assert False, 'Scratchpad area!'

AssertionError: Scratchpad area!

In [11]:
conn.nebula.l1ic_size.delete_many({})

In [ ]:
print(dir(conn))

['HOST', 'PORT', '_BaseObject__codec_options', '_BaseObject__read_concern', '_BaseObject__read_preference', '_BaseObject__write_concern', '_MongoClient__default_database_name', '_MongoClient__init_kwargs', '_MongoClient__kill_cursors_queue', '_MongoClient__lock', '_MongoClient__options', '_MongoClient__start_session', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_cleanup_cursor', '_close_cursor_now', '_close_cursor_soon', '_constructor_args', '_database_default_options', '_duplicate', '_encrypter', '_end_sessions', '_ensure_se

In [ ]:
print(conn.list_database_names())

['admin', 'config', 'local', 'nebula', 'tutorial', 'ussim']


In [ ]:
print(conn.nebula.list_collection_names())

['smoke', 'experiments']


In [ ]:
conn.nebula.experiments.find_one().keys()

dict_keys(['_id', 'sha', 'branch', 'exitcode', 'stats', 'config', 'runpath', 'pipeline', 'script', 'cmdline', 'date', 'time'])

In [ ]:
conn.nebula.experiments.find_one().get('stats').get('cycle')

2301

In [ ]:
conn.nebula.smoke.find_one().get('exitcode')

0